In [44]:
import cv2
import cv2 as cv
import numpy as np
import mediapipe as mp

In [45]:
# Coordinates of eye points that will be used to calculate the EAR (Eye Aspect Ratio)
p_olho_esq = [385, 380, 387, 373, 362, 263]
p_olho_dir = [160, 144, 158, 153, 33, 133]
p_olhos = p_olho_esq + p_olho_dir

In [46]:
def calculo_ear(face, p_olho_dir: list, p_olho_esq: list) -> float:
    """
    Calculate the EAR (Eye Aspect Ratio) of the face.
    :param face: face mesh returned by MediaPipe
    :param p_olho_dir: list of eye points of the right eye
    :param p_olho_esq: list of eye points of the left eye
    :return: EAR of the face
    """
    
    face_array = np.array([[coord.x, coord.y] for coord in face])
    
    face_esq = face_array[p_olho_esq, :]
    face_dir = face_array[p_olho_dir, :]
    
    ear_esq = (np.linalg.norm(face_esq[0]-face_esq[1]) + np.linalg.norm(face_esq[2]-face_esq[3])) / (2*np.linalg.norm(face_esq[4]-face_esq[5]))
    
    ear_dir = (np.linalg.norm(face_dir[0]-face_dir[1]) + np.linalg.norm(face_dir[2]-face_dir[3])) / (2*np.linalg.norm(face_dir[4]-face_dir[5]))
    
    return (ear_esq + ear_dir) / 2

In [47]:
# Defining:
# 1. The drawing utilities of MediaPipe
# 2. The FaceMesh model
# 3. The video capture
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
cap = cv.VideoCapture(0)


with mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    # refine_landmarks=True ## Este parâmetro pode ser utilizado para detectar íris
) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Ignoring empty camera frame.")
            continue

        height, width, _ = frame.shape
        
        # Flip the image horizontally for a later selfie-view display
        frame = cv.cvtColor(cv.flip(frame, 1), cv.COLOR_BGR2RGB)
        
        # Send the frame to MediaPipe FaceMesh and get the result
        results = face_mesh.process(frame)
        
        #Converting the image back to BGR
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        
        if results.multi_face_landmarks:
            # Iterating over the landmarks detected
            for face_landmarks in results.multi_face_landmarks:
                # Drawing the face landmarks on the frame                
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=1),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
                )
                        
                face = face_landmarks.landmark
                        
                for id_coord, coord_xyz in enumerate(face):
                    # Checking if the landmark is an eye point
                    if id_coord in p_olhos:
                        coord_cv = mp_drawing._normalized_to_pixel_coordinates(coord_xyz.x, coord_xyz.y, width, height)
                    
                        cv.circle(frame, coord_cv, 2, (255, 0, 0), -1)
                
                ear = calculo_ear(face, p_olho_dir, p_olho_esq)
                
                # Provisionally displaying the EAR on the screen                
                cv2.rectangle(frame, (0,1), (290, 140), (58, 58, 55), -1)
                cv2.putText(frame, f'EAR: {round(ear, 2)}', (1, 24), cv2.FONT_HERSHEY_DUPLEX, 0.9, (255, 255, 255), 2)
                
        cv.imshow('MediaPipe FaceMesh', frame)
        
        # Break the loop when the 'q' key is pressed
        if cv.waitKey(5) & 0xFF == ord('q'):
            break   

# Release the camera and destroy all windows
cap.release()
cv.destroyAllWindows()

<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'> <class 'list'> <class 'list'>
<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'> <class 'list'> <class 'list'>
<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'> <class 'list'> <class 'list'>
<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'> <class 'list'> <class 'list'>
<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'> <class 'list'> <class 'list'>
<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'> <class 'list'> <class 'list'>
<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'> <class 'list'> <class 'list'>
<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'> <class 'list'> <class 'list'>
<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'> <class 'list'> <class 'list'>
<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'> <class 'list'> <class 'list'>
<class 'google.proto